In [1]:
import pandas as pd
import os
import json

#chemin_archive ="/users/2023/ds1/share/CORD-19"
chemin_archive = "D:/archive"
chemin_tables=f'{chemin_archive}/Kaggle/target_tables'
elements = os.listdir(chemin_tables)
dossiers = [element for element in elements if os.path.isdir(os.path.join(chemin_tables, element))]
for dossier in dossiers[1:-1]:
    theme=(dossier[2:].replace("_"," ")).upper()   # le theme est le nom du dossier
    print(f'{theme=}')

theme='POPULATION'
theme='RELEVANT FACTORS'
theme='PATIENT DESCRIPTIONS'
theme='MODELS AND OPEN QUESTIONS'
theme='MATERIALS'
theme='DIAGNOSTICS'
theme='THERAPEUTICS INTERVENTIONS AND CLINICAL STUDIES'
theme='RISK FACTORS'


In [2]:
DF=pd.read_csv(f'{chemin_archive}/metadata.csv')
DF.columns

C:\Users\Zeynep\AppData\Local\Temp\ipykernel_3544\2389843411.py:1: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  DF=pd.read_csv(f'{chemin_archive}/metadata.csv')


Index(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'pmcid', 'pubmed_id',
       'license', 'abstract', 'publish_time', 'authors', 'journal', 'mag_id',
       'who_covidence_id', 'arxiv_id', 'pdf_json_files', 'pmc_json_files',
       'url', 's2_id'],
      dtype='object')

In [3]:
chemin1 = f'{chemin_archive}/document_parses/pmc_json'
elements1 = os.listdir(chemin1)
no_affiliation_dans_pmc=0
affiliation_dans_pmc=0
for element in elements1:
    with open(f'{chemin1}/{element}', 'r') as f:
        data = json.load(f)
        a=data['metadata']['authors']
        if len(a)!=0:
            for i in range(len(a)):
                if len(data['metadata']['authors'][i]['affiliation'])!=0:
                    affiliation_dans_pmc+=1
                else:
                    no_affiliation_dans_pmc+=1
print(f'{affiliation_dans_pmc=}')
print(f'{no_affiliation_dans_pmc=}')

affiliation_dans_pmc=0
no_affiliation_dans_pmc=2401600


In [4]:
len(elements1) == len(DF[DF['pmc_json_files'].notnull()])

True

In [5]:
no_pmc_files=[]
pmc_files_load=0
pmc_files=DF[DF['pmc_json_files'].notnull()]['pmc_json_files']
for file in pmc_files:
    try:
        with open(f'{chemin_archive}/{file}', 'r') as f:
            data = json.load(f)
            pmc_files_load+=1
    except:
        no_pmc_files.append(file)
print(f'{pmc_files_load=}')
print(f'{no_pmc_files=}')

pmc_files_load=315742
no_pmc_files=[]


In [6]:
print("nombre d'article avec un pmc_files et un pdf_files ="
      ,len(DF[DF['pdf_json_files'].notnull() & DF["pmc_json_files"].notnull()]))
print("nombre d'article avec uniquement pmc_files =",len(DF[DF['pdf_json_files'].isnull() & DF["pmc_json_files"].notnull()]))
print("nombre d'article avec uniquement pdf_files =",len(DF[DF['pdf_json_files'].notnull() & DF["pmc_json_files"].isnull()]))

nombre d'article avec un pmc_files et un pdf_files = 299609
nombre d'article avec uniquement pmc_files = 16133
nombre d'article avec uniquement pdf_files = 74157


In [7]:
DF_essais=DF[DF['pmc_json_files'].notnull()].reset_index()
for k in range(len(DF_essais)):
    auteurs_metadata=[]
    auteurs_pmc=[]
    ligne=DF_essais['authors'][k]
    file=DF_essais['pmc_json_files'][k]
    if type(ligne)==str:
        for author in ligne.split('; '):
            auteurs_metadata.append(author)
    with open(f'{chemin_archive}/{file}','r') as f:
        data=json.load(f)
        L=data['metadata']['authors']
        if len(L)!=0:
            for i in range(len(L)):
                name=L[i]['last']+', '+L[i]['first']
                auteurs_pmc.append(name)
    if len(auteurs_pmc)!=len(auteurs_metadata):
        print("liste d'auteurs dans metadata.csv pour un article =",'\n',ligne,'\n')
        print("le fichier pmc_json_files =",file,'\n')
        print("liste d'auteurs dans le fichier pmc_json_files pour un article =",'\n',"; ".join(auteurs_pmc))
        break

liste d'auteurs dans metadata.csv pour un article = 
 Froissart, Remy; Roze, Denis; Uzest, Marilyne; Galibert, Lionel; Blanc, Stephane; Michalakis, Yannis 

le fichier pmc_json_files = document_parses/pmc_json/PMC1054884.xml.json 

liste d'auteurs dans le fichier pmc_json_files pour un article = 
 Froissart, Remy; Roze, Denis; Uzest, Marilyne; Galibert, Lionel; Blanc, Stephane; Michalakis, Yannis; Hull, Roger


In [11]:
#Si un article a un fichier pmc_json_files et un fichier pdf_json_files, il n'y a pas les même auteurs dans les 2 fichiers.
DF_essais=DF[DF['pdf_json_files'].notnull() & DF["pmc_json_files"].notnull()].reset_index()
for k in range(len(DF_essais)):
    auteurs_pdf=[]
    auteurs_pmc=[]
    file_pmc=DF_essais['pmc_json_files'][k]
    file_pdf=DF_essais['pdf_json_files'][k]
    with open(f'{chemin_archive}/{file_pmc}','r') as f:
        data=json.load(f)
        L=data['metadata']['authors']
        if len(L)!=0:
            for i in range(len(L)):
                name=L[i]['last']+', '+L[i]['first']
                auteurs_pmc.append(name)
    liste_file=file_pdf.split('; ')
    for fil in liste_file:
        try:
            with open(f'{chemin_archive}/{fil}', 'r') as f:
                data = json.load(f)
                L=data['metadata']['authors']
                if len(L)!=0:
                    for i in range(len(L)):
                        name=L[i]['last']+', '+L[i]['first']
                        auteurs_pdf.append(name)
        except:
            print(file_pdf)
    if len(auteurs_pmc)!=len(auteurs_pdf):
        print(file_pmc)
        print(auteurs_pmc)
        print(file_pdf)
        print(auteurs_pdf)
        print(k)
        break

document_parses/pmc_json/PMC59574.xml.json
['Fagan, Karen', 'McMurtry, Ivan', 'Rodman, David']
document_parses/pdf_json/348055649b6b8cf2b9a376498df9bf41f7123605.json
[]
3


In [12]:
#si un article a plusieurs pdf_json_files, il n'y a pas les même données (auteurs, emails, affiliation) dans ces fichiers.
DF_essais=DF[DF['pdf_json_files'].notnull()]
for file_pdf in DF_essais['pdf_json_files'][:100]:
    if '; ' in file_pdf:
        liste_file=file_pdf.split('; ')
        print("nombre de pdf_json_files = ",len(liste_file))
        for fil in liste_file:
            auteurs=[]
            try:
                with open(f'{chemin_archive}/{fil}', 'r') as f:
                    data = json.load(f)
                    L=data['metadata']['authors']
                    if len(L)!=0:
                        for i in range(len(L)):
                            name=L[i]['last']+', '+L[i]['first']
                            auteurs.append(name)              
            except:
                print(fil,'e')
            print(auteurs)

nombre de pdf_json_files =  2
['Malanoski, Anthony', 'Lin, Baochuan', 'Wang, Zheng', 'Schnur, Joel', 'Stenger, David']
[]
nombre de pdf_json_files =  2
['Mccrate, Nina', 'Varner, Mychel', 'Kim, Kenneth', 'Nagan, Maria', 'Uuu, [', 'Yarian, M', 'Marszalek, E', 'Sochacka, A', 'Malkiewicz, R', 'Guenther, A', 'Miskiewicz, P']
['Mccrate, Nina', 'Varner, Mychel', 'Kim, Kenneth', 'Nagan, Maria']
nombre de pdf_json_files =  2
['Kutyavin, Igor', 'Milesi, Dave', 'Belousov, Yevgeniy', 'Podyminogin, Mikhail', 'Vorobiev, Alexei', 'Gorn, Vladimir', 'Lukhtanov, Eugeny', 'Vermeulen, Nicolaas', 'Mahoney, Walt']
[]


In [13]:
files_pdf=[]
DF_essais=DF[DF['pdf_json_files'].notnull()].reset_index()
for file_pdf in DF_essais['pdf_json_files']:
    liste_file=file_pdf.split('; ')
    for fil in liste_file:
        files_pdf.append(fil)
print(len(files_pdf))
print(len(set(files_pdf)))
chemin2 = f'{chemin_archive}/document_parses/pdf_json'
elements2 = os.listdir(chemin2)
print(len(elements2))

401270
401214
401212


In [14]:
pdf_files_load=0
no_pdf_files=[]
for file in files_pdf: #regarder précédent pour files_pdf
    try:
        with open(f'{chemin_archive}/{file}', 'r') as f:
            data = json.load(f)
            pdf_files_load+=1
    except:
        no_pdf_files.append(file)
print(f'{pdf_files_load=}')
print(f'{no_pdf_files=}')

pdf_files_load=401268
no_pdf_files=['document_parses/pdf_json/c6574ba0edcefbbf9b26914657f49969d2b0a8d6.json', 'document_parses/pdf_json/345c97e357144264e93dacb9f2b80005b26cb750.json']


NameError: name 'pdf_files' is not defined

In [15]:
print('c6574ba0edcefbbf9b26914657f49969d2b0a8d6.json' in elements2)
print('345c97e357144264e93dacb9f2b80005b26cb750.json' in elements2)

False
False


In [16]:
import pandas as pd
import os
import json

chemin_archive ="D:/archive"
chemin_tables=f'{chemin_archive}/Kaggle/target_tables'
elements = os.listdir(chemin_tables)
dossiers = [element for element in elements if os.path.isdir(os.path.join(chemin_tables, element))]

chemin1 = f'{chemin_archive}/document_parses/pmc_json'
elements1 = os.listdir(chemin1)

chemin2 = f'{chemin_archive}/document_parses/pdf_json'
elements2 = os.listdir(chemin2)

DF=pd.read_csv(f'{chemin_archive}/metadata.csv')

C:\Users\Zeynep\AppData\Local\Temp\ipykernel_3544\250716358.py:16: DtypeWarning: Columns (1,4,5,6,13,14,15,16) have mixed types. Specify dtype option on import or set low_memory=False.
  DF=pd.read_csv(f'{chemin_archive}/metadata.csv')


In [17]:
Articles=[]
Study_Article=[]
for dossier in dossiers[1:-1]:
    chemin = f'{chemin_tables}/{dossier}'
    elements = os.listdir(chemin)
    for element in elements :
        df=pd.read_csv(f'{chemin}/{element}')
        try :
            for i in range(len(df)):
                study_article=(df['Study Type'][i],df['Study'][i])
                Articles.append(df['Study'][i])
                Study_Article.append(study_article)
        except:
            print(f"Le fichier {element} n'a pas de studytype")
print(len(Study_Article))
#print(Study_Article)
print(len(Articles))
print(len(set(Articles)))

Le fichier Are there studies about phenotypic change_.csv n'a pas de studytype
Le fichier How can we measure changes in COVID-19_s behavior in a human host as the virus evolves over time_.csv n'a pas de studytype
Le fichier Studies to monitor potential adaptations.csv n'a pas de studytype
Le fichier What is known about adaptations (mutations) of the virus_.csv n'a pas de studytype
2488
2488
1044


In [18]:
for Article in set(Articles):
    types=[]
    for i in range(len(Study_Article)):
        if Study_Article[i][1]==Article:
            try:
                s=Study_Article[i][0].upper()
            except:
                s=Study_Article[i][0]
            types.append(s)
    l=len(list(set(types)))
    if l>=2:
        print(list(set(types)))
        print(Article)
        break
# Nous avons découvert qu'un article pouvait avoir 2 StudyType (= type de publications)

['RETROSPECTIVE OBSERVATIONAL STUDY', 'CASE SERIES']
Epidemiological and clinical characteristics of COVID-19 in adolescents and young adults
